In [ ]:
!pip install torch torchvision torchaudio
!pip install pandas
!pip install mlflow

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import init
import pandas as pd
import numpy as np
import random
import os

In [ ]:
# Add these imports at the top
import math
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [ ]:
# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load your Banking77 dataset
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Load the dataset
train_df = pd.read_csv('/content/drive/MyDrive/Banking77_Project/data/train.csv')
print(f"Training data shape: {train_df.shape}")
print(train_df.head())

In [ ]:
# Extract text data
text = " ".join(train_df['text'].astype(str).tolist())
print(f"Total characters in dataset: {len(text)}")
print(text[:1000])  # Show first 1000 characters

In [ ]:
# Create character-level vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f"Vocabulary size: {vocab_size}")

In [ ]:

# Create mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
# Test encoding/decoding
test_str = "banking query"
encoded = encode(test_str)
decoded = decode(encoded)
print(f"Original: {test_str}")
print(f"Encoded: {encoded}")
print(f"Decoded: {decoded}")


In [ ]:
# Add this after your encode/decode functions
def text_to_tokens(text, block_size):
    encoded = encode(text)
    if len(encoded) > block_size:
        encoded = encoded[:block_size]
    else:
        encoded = encoded + [0] * (block_size - len(encoded))
    return torch.tensor(encoded, dtype=torch.long).unsqueeze(0)


In [ ]:
# Encode the entire text dataset
data = torch.tensor(encode(text), dtype=torch.long)
print(f"Data shape: {data.shape}")

# Split into training and validation sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print(f"Training data size: {len(train_data)}")
print(f"Validation data size: {len(val_data)}")

In [ ]:
batch_size = 64  # Increased batch size
block_size = 256  # Longer context
max_iters = 10000  # More training iterations
eval_interval = 500
learning_rate = 1e-4  # Lower learning rate for better stability
eval_iters = 250
head_size = 64  # Larger head size
n_embed = 512  # Increased embedding dimension
n_head = 8
n_layer = 8
dropout = 0.1  # Reduced dropout
num_experts = 8
top_k = 2

In [ ]:
# Data loading function
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [ ]:
# Loss estimation function
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Improved model components with better initialization
class ImprovedHead(nn.Module):
    """Improved self-attention head with better initialization"""
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)

        # Better initialization
        init.kaiming_uniform_(self.key.weight)
        init.kaiming_uniform_(self.query.weight)
        init.kaiming_uniform_(self.value.weight)

        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out


In [ ]:
class MultiHeadAttention(nn.Module):
    """Multiple heads of self-attention in parallel"""
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class ImprovedExpert(nn.Module):
    """Improved MLP expert with better architecture"""
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.GELU(),  # Changed to GELU for better performance
            nn.LayerNorm(4 * n_embed),  # Added layer norm
            nn.Dropout(dropout),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

        # Better initialization
        for layer in self.net:
            if isinstance(layer, nn.Linear):
                init.kaiming_uniform_(layer.weight)

    def forward(self, x):
        return self.net(x)


In [ ]:
class NoisyTopkRouter(nn.Module):
    """Noisy top-k gating network"""
    def __init__(self, n_embed, num_experts, top_k):
        super(NoisyTopkRouter, self).__init__()
        self.top_k = top_k
        self.topkroute_linear = nn.Linear(n_embed, num_experts)
        self.noise_linear = nn.Linear(n_embed, num_experts)

    def forward(self, mh_output):
        logits = self.topkroute_linear(mh_output)
        noise_logits = self.noise_linear(mh_output)
        noise = torch.randn_like(logits) * F.softplus(noise_logits)
        noisy_logits = logits + noise
        top_k_logits, indices = noisy_logits.topk(self.top_k, dim=-1)
        zeros = torch.full_like(noisy_logits, float('-inf'))
        sparse_logits = zeros.scatter(-1, indices, top_k_logits)
        router_output = F.softmax(sparse_logits, dim=-1)
        return router_output, indices

In [ ]:
class ImprovedSparseMoE(nn.Module):
    """Improved Sparse Mixture of Experts module"""
    def __init__(self, n_embed, num_experts, top_k):
        super(ImprovedSparseMoE, self).__init__()
        self.router = NoisyTopkRouter(n_embed, num_experts, top_k)
        self.experts = nn.ModuleList([ImprovedExpert(n_embed) for _ in range(num_experts)])
        self.top_k = top_k

    def forward(self, x):
        gating_output, indices = self.router(x)
        final_output = torch.zeros_like(x)
        flat_x = x.view(-1, x.size(-1))
        flat_gating_output = gating_output.view(-1, gating_output.size(-1))

        for i, expert in enumerate(self.experts):
            expert_mask = (indices == i).any(dim=-1)
            flat_mask = expert_mask.view(-1)

            if flat_mask.any():
                expert_input = flat_x[flat_mask]
                expert_output = expert(expert_input)
                gating_scores = flat_gating_output[flat_mask, i].unsqueeze(1)
                weighted_output = expert_output * gating_scores
                final_output[expert_mask] += weighted_output.squeeze(1)

        return final_output

In [ ]:
class ImprovedBlock(nn.Module):
    """Improved Transformer block"""
    def __init__(self, n_embed, n_head, num_experts, top_k):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.smoe = ImprovedSparseMoE(n_embed, num_experts, top_k)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Residual connections with better scaling
        x = x + self.dropout(self.sa(self.ln1(x)))
        x = x + self.dropout(self.smoe(self.ln2(x)))
        return x


In [ ]:
class ImprovedSparseMoELanguageModel(nn.Module):
    """Improved Sparse MoE Language Model"""
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[ImprovedBlock(n_embed, n_head, num_experts, top_k) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

        # Better initialization
        init.kaiming_uniform_(self.token_embedding_table.weight)
        init.kaiming_uniform_(self.position_embedding_table.weight)
        init.kaiming_uniform_(self.lm_head.weight)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [ ]:
# Initialize improved model
model = ImprovedSparseMoELanguageModel()
model = model.to(device)

# Print number of parameters
print(f"{sum(p.numel() for p in model.parameters())/1e6:.2f} M parameters")

# Create optimizer with weight decay
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)

# Corrected scheduler without the 'verbose' argument
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=200)

In [ ]:
# Initialize model
def kaiming_init_weights(m):
    if isinstance(m, nn.Linear):
        init.kaiming_normal_(m.weight)

model = SparseMoELanguageModel()
model.apply(kaiming_init_weights)
model = model.to(device)

# Print number of parameters
print(f"{sum(p.numel() for p in model.parameters())/1e6:.2f} M parameters")

# Create optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_iters, eta_min=learning_rate/10)

In [ ]:
# Training loop
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    scheduler.step()


In [ ]:
# Generate a sample from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_tokens = model.generate(context, max_new_tokens=2000)[0].tolist()

# Decode the generated tokens back into text and print
generated_text = decode(generated_tokens)
print(generated_text)
print("Generated text:")
print(generated_text)

In [ ]:
# Save the model
torch.save(model.state_dict(), '/content/drive/MyDrive/Banking77_Project/moe_model.pth')
print("Model saved!")

In [ ]:
print("\n Testing Sample Predictions...")

model.eval()
sample_texts = [
    "what is my account balance",
    "transfer money to my friend",
    "my card is not working",
    "I want to apply for a loan"
]


In [ ]:
with torch.no_grad():
    for text in sample_texts:
        # Tokenize
        token_ids = tokenizer.tokenize(text).unsqueeze(0).to(device)
        attention_mask = (token_ids != 0).float()

        # Predict
        outputs = model(token_ids, attention_mask)
        predicted_id = torch.argmax(outputs['logits'], dim=1).item()
        confidence = torch.softmax(outputs['logits'], dim=1).max().item()

        # Get expert usage
        expert_weights = outputs['routing_weights'][0].cpu().numpy()
        top_experts = [f"E{i+1}({w:.2f})" for i, w in enumerate(expert_weights) if w > 0.01]

        print(f"Text: '{text}'")
        print(f"  → Predicted: {id2label[predicted_id]} (confidence: {confidence:.3f})")
        print(f"  → Experts used: {', '.join(top_experts)}")
        print()